In [114]:
#import libraries needed for this proces
import requests
from datetime import datetime
import numpy
import time

In [ ]:

#function to create an API request based on URL and  parameters
def get_request(url, parameters=None):
    print("Your request is being processed.")
    response = requests.get(url=url, params=parameters)
    if response:
        return response.json()
    else:
        print("Request unsuccessful, retrying in 5 seconds.")
        time.sleep(5)
        return get_request(url, parameters)
    time.sleep(5)



#using the function to get all data id's which include appid and app name 
url = "https://api.steampowered.com/ISteamApps/GetAppList/v2/"
parameters = {"request": "all"}
data_ids = get_request(url, parameters=parameters)
print("Your request has been processed successfully, call upon 'data_ids' to preview the full list of IDs and names of all currently available products on Steam as of ", datetime.now().strftime("%d/%m/%Y %H:%M:%S"), ".")

data_ids

In [ ]:
#Code to collect all app id's in one library
app_ids = []
print("Creating a list of all IDs currently available on Steam")
for item in data_ids['applist']['apps']:
    app_ids.append(item['appid'])

print("List created sucessfully, the total number of IDs available on Steam is:", len(app_ids),".")
print("Call upon 'app_ids' to preview the full list of extracted IDs.")


In [ ]:
#code that collects the details of 1 app by appid  
url = "http://store.steampowered.com/api/appdetails/"
parameters = {"appids": "1337020"}
app_data = get_request(url, parameters=parameters)
app_data

In [180]:
#it finaly loops
#however it gives some succes and some false values. so we need to build in a check if succes = false -> then next. if succes = True -> write the data to a Csv 
import json

for item in app_ids:
    url_appdetails= "http://store.steampowered.com/api/appdetails/"
    parameters = {"appids": {item}}
    app_details = get_request(url_appdetails, parameters=parameters)
    
    print(app_details)
    time.sleep(5)
    


Your request is being processed.
{'1360782': {'success': False}}
Your request is being processed.
{'662172': {'success': False}}


KeyboardInterrupt: 

In [ ]:
#try number XYZ (created a function with a while loop to itterate over it but it didnt' work)
app_dict = []
app_data = {}

counter = 1 

while counter <= 5:
    
    for i in app_ids: 
        url = "http://store.steampowered.com/api/appdetails/"
        parameters = {"appids": {i}}
        app_data = get_request(url, parameters=parameters)       
        app_data["app_id"] = i
        counter += 1
    counter + 1
        
    

In [ ]:
#another attempt

 
for i in app_ids: 
    url = "http://store.steampowered.com/api/appdetails/"
    parameters = {"appids": {i}}
    app_data = get_request(url, parameters=parameters)       
    temp = app_data.get(i)
    print(temp)
    
    
    col_data = []


    #ol_data["app_id"] = i
    #col_data["name"]
